In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
 if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.Sparkle24Filled />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <React.Fragment key={index}>
              <div
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
              {message.role === "user" && index === 0 && isFirstQuery && (
                <div className={styles.chartContainer}>
                  <StockChart data={stockData} />
                </div>
              )}
              {message.subgraphResults && (
                <div className={styles.subgraphMessage}>
                  <SubgraphResults results={message.subgraphResults} />
                </div>
              )}
            </React.Fragment>
          ))}
        </div>

In [ ]:
from typing import Dict, List
from langgraph.graph import StateGraph, END

# Define your functions
def vector_db_search(query: str) -> str:
    # Implement your vector database search here
    return f"Vector DB result for: {query}"

def query_expansion(query: str) -> List[str]:
    # Implement your query expansion logic here
    return [f"{query} expanded 1", f"{query} expanded 2"]

def api_call(query: str) -> str:
    # Implement your API call here
    return f"API result for: {query}"

# Define the routing logic
def route_query(state):
    query = state['query']
    if len(query.split()) > 3:  # Simple routing logic, adjust as needed
        return "query_expansion"
    else:
        return "vector_db_search"

# Define the query expansion and API call logic
def expand_and_call(state):
    expanded_queries = query_expansion(state['query'])
    results = []
    for eq in expanded_queries:
        results.append(api_call(eq))
    return {"expanded_results": results}

# Define the aggregation logic
def aggregate_results(state):
    if "expanded_results" in state:
        # Aggregate expanded results
        combined = " ".join(state["expanded_results"])
        return {"response": f"Aggregated results: {combined}"}
    else:
        # Return vector DB result
        return {"response": state["result"]}

# Create the graph
workflow = StateGraph(name="QueryProcessor")

# Add nodes
workflow.add_node("route", route_query)
workflow.add_node("vector_db_search", lambda state: {"result": vector_db_search(state['query'])})
workflow.add_node("expand_and_call", expand_and_call)
workflow.add_node("aggregate", aggregate_results)

# Add edges
workflow.add_edge("route", "vector_db_search")
workflow.add_edge("route", "expand_and_call")
workflow.add_edge("vector_db_search", "aggregate")
workflow.add_edge("expand_and_call", "aggregate")
workflow.add_edge("aggregate", END)

# Set the entrypoint
workflow.set_entry_point("route")

# Compile the graph
app = workflow.compile()

# Example usage
query = "What is the capital of France?"
result = app.invoke({"query": query})
print(result['response'])

In [ ]:
from PyPDF2 import PdfReader

def read_pdf_comments(pdf_path):
    """
    Read comments/annotations from a PDF file using PyPDF2.
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        list: List of dictionaries containing comment information
    """
    all_comments = []
    
    try:
        # Open the PDF file
        reader = PdfReader(pdf_path)
        
        # Iterate through each page
        for page_num, page in enumerate(reader.pages, 1):
            if '/Annots' in page:
                # Get annotations from the page
                annotations = page['/Annots']
                
                # Some annotations might be indirect references
                if annotations is not None:
                    for annot in annotations:
                        # Resolve indirect object reference
                        annot_obj = annot.get_object()
                        
                        comment_info = {
                            'page': page_num,
                            'type': annot_obj.get('/Subtype', ''),
                            'content': annot_obj.get('/Contents', ''),
                            'author': annot_obj.get('/T', ''),  # T is for author/title
                            'created': annot_obj.get('/CreationDate', ''),
                            'modified': annot_obj.get('/M', ''),  # M is for modification date
                            'rect': annot_obj.get('/Rect', [])
                        }
                        
                        # Add additional details for specific annotation types
                        if comment_info['type'] == '/Highlight':
                            comment_info['color'] = annot_obj.get('/C', [])
                            comment_info['quadpoints'] = annot_obj.get('/QuadPoints', [])
                            
                        all_comments.append(comment_info)
        
        return all_comments
        
    except Exception as e:
        print(f"Error reading PDF: {str(e)}")
        return []

def print_comments_summary(comments):
    """
    Print a formatted summary of all comments found in the PDF.
    
    Args:
        comments (list): List of comment dictionaries returned by read_pdf_comments
    """
    if not comments:
        print("No comments found in the PDF.")
        return
        
    for i, comment in enumerate(comments, 1):
        print(f"\nComment #{i}:")
        print(f"Page: {comment['page']}")
        print(f"Type: {comment['type']}")
        if comment['content']:
            print(f"Content: {comment['content']}")
        if comment['author']:
            print(f"Author: {comment['author']}")
        if comment['created']:
            print(f"Created: {comment['created']}")
        if comment['modified']:
            print(f"Modified: {comment['modified']}")
        print(f"Location: {comment['rect']}")
        
        # Print highlight-specific information
        if comment['type'] == '/Highlight':
            if 'color' in comment:
                print(f"Color: {comment['color']}")

# Example usage
if __name__ == "__main__":
    pdf_path = "sample.pdf"
    comments = read_pdf_comments(pdf_path)
    print_comments_summary(comments)